# Teste Engenheiro de Dados da Semantix

Questões: Responda as seguintes questões devem ser desenvolvidas em Spark utilizando a sua linguagem de preferência.
1. Número de hosts únicos.
2. O total de erros 404.
3. Os 5 URLs que mais causaram erro 404.
4. Quantidade de erros 404 por dia.
5. O total de bytes retornados

## Primeiro eu importo as bibliotecas

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import functions 
from pyspark.sql import types 
from datetime import date, timedelta, datetime
import time

In [2]:
# Nesta celula o SparkSession é inicializado com o nome de sc
sc = SparkSession.builder.appName("PysparkExample").config("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g").config ("spark.sql.execution.arrow.enabled", "true") .getOrCreate()

In [3]:
# Aqui leio os dois arquivos e add o segundo no final do primeiro e separo as colunas por "espaço"
df = sc.read.option("header", "False") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .csv(["access_log_Jul95","access_log_Aug95"])

In [4]:
# Com esta separação ficam 8 colunas, sendo que algumas poderiam ser excluídas 
print((df.count(), len(df.columns)))

(3461613, 8)


In [5]:
df.show(10)

+--------------------+---+---+--------------------+------+--------------------+---+----+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6| _c7|
+--------------------+---+---+--------------------+------+--------------------+---+----+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200|6245|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|3985|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4085|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|   0|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4179|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|   0|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|   0|
|     205.212.115.106|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|3985|
|         d104.aa.net

# Uma vez organizado o DataFrame, responderei as perguntas utilizando queries do SQL dentro do pyspark

### Q1: Número de hosts únicos

In [6]:
df.createOrReplaceTempView("Semantix")
sc.sql("select COUNT(DISTINCT _c0) AS Hosts_unicos from Semantix").show() 

+------------+
|Hosts_unicos|
+------------+
|      137979|
+------------+



## _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_

### Q2: O total de erros 404.

In [7]:
sc.sql("select count(_c6) AS Total_erros_404 from Semantix WHERE _c6 = 404").show()

+---------------+
|Total_erros_404|
+---------------+
|          20871|
+---------------+



## _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_

### Q3: Os 5 URLs que mais causaram erro 404.

In [8]:
sc.sql("select _c0 AS URL, count(_c6) AS URLs_Max_404 from Semantix WHERE _c6 = 404 GROUP BY URL ORDER BY URLs_Max_404 desc").show(5)

+--------------------+------------+
|                 URL|URLs_Max_404|
+--------------------+------------+
|hoohoo.ncsa.uiuc.edu|         251|
|piweba3y.prodigy.com|         156|
|jbiagioni.npt.nuw...|         132|
|piweba1y.prodigy.com|         114|
|www-d4.proxy.aol.com|          91|
+--------------------+------------+
only showing top 5 rows



## _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_

### Q4: Quantidade de erros 404 por dia.

In [9]:
# Para responder esta questão eu dividi a _c3 que tem o dia, mês, ano, hora e zona, utilizando ":" como separador e pequei
# apenas a primeira coluna do resultado, que é justamente a que tem dia, mês e ano
split_col = functions.split(df['_c3'], ':')
df2 = df.withColumn('Days', split_col.getItem(0))

In [10]:
df2.show(10)

+--------------------+---+---+--------------------+------+--------------------+---+----+------------+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6| _c7|        Days|
+--------------------+---+---+--------------------+------+--------------------+---+----+------------+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200|6245|[01/Jul/1995|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|3985|[01/Jul/1995|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4085|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|   0|[01/Jul/1995|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4179|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|   0|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200

In [11]:
df2.createOrReplaceTempView("Semantix2")

In [12]:
sc.sql("select * from Semantix2").show()

+--------------------+---+---+--------------------+------+--------------------+---+-----+------------+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6|  _c7|        Days|
+--------------------+---+---+--------------------+------+--------------------+---+-----+------------+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200| 6245|[01/Jul/1995|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200| 3985|[01/Jul/1995|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200| 4085|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|    0|[01/Jul/1995|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200| 4179|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|    0|[01/Jul/1995|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/co

In [13]:
sc.sql("select Days, count(_c6) AS Quant_erros_404 from Semantix2 WHERE _c6 = 404 GROUP BY Days ORDER BY Quant_erros_404 desc").show(5)

+------------+---------------+
|        Days|Quant_erros_404|
+------------+---------------+
|[06/Jul/1995|            640|
|[19/Jul/1995|            638|
|[30/Aug/1995|            571|
|[07/Jul/1995|            569|
|[07/Aug/1995|            537|
+------------+---------------+
only showing top 5 rows



## _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_

### Q5: O total de bytes retornados.

In [14]:
sc.sql("select sum(_c7) AS Total_bytes from Semantix2").show()

+---------------+
|    Total_bytes|
+---------------+
|6.5524319796E10|
+---------------+

